**IOC 23 – ARP Spoofing Attack**
Category: Network Activity Indicator
IOC Type: Layer 2 ARP Cache Poisoning (Man-in-the-Middle Setup)
Telemetry Origin: Packet Capture (Wireshark) and Network Device Logs

**1. Attacker Analogy**
Think of a building where each office connects by internal mail. Normally, everyone addresses letters by recipient name, and mail is routed accordingly. But what if someone inside the building quietly swaps the address labels on envelopes? Suddenly, the mail intended for the CEO’s office is routed to the attacker’s cubicle. They read it, copy it, and forward it along.
This is ARP spoofing: the attacker intercepts data by convincing the network that their machine is the destination for traffic intended for another. They manipulate the internal addressing system – in this case, the ARP (Address Resolution Protocol) table – to divert traffic through their device. The result? They become a man-in-the-middle (MitM), capable of eavesdropping, modifying, or redirecting packets at will.

**2. IOC / Source of Telemetry (Observable and Raw Data)**
Observable Artifact:

Wireshark capture shows unsolicited ARP replies (gratuitous ARP replies) originating from a suspicious MAC address, falsely claiming that it owns the IP address of the default gateway — the network’s router or primary exit point. These repeated claims are the attacker’s way of tricking systems into misrouting their traffic through the attacker’s device.

Simulated Raw Metadata (Wireshark):

Frame 42112: 60 bytes on wire (480 bits), 60 bytes captured  
Ethernet II, Src: aa:bb:cc:dd:ee:ff (attacker MAC), Dst: ff:ff:ff:ff:ff:ff (broadcast)  
Address Resolution Protocol (reply)  
    Hardware type: Ethernet (1)  
    Protocol type: IPv4 (0x0800)  
    Opcode: reply (2)  
    Sender MAC address: aa:bb:cc:dd:ee:ff  
    Sender IP address: 192.168.1.1 (default gateway)  
    Target MAC address: ff:ff:ff:ff:ff:ff  
    Target IP address: 192.168.1.10 (victim host)  
Telemetry Source:
Primary: Wireshark capture during incident response

Supplementary: Switch logs showing frequent ARP updates for gateway IP

Softened Interpretation:
The attacker’s machine is repeatedly shouting, “I’m the gateway! Trust me!” – and the victim system, seeing no reason to doubt, believes it.

**3. Triage Framework Declaration**
Triage Type: Network Activity Indicator
Canonical Toolset and Required Sequence:

Nmap Scan: Identify exposed hosts responding to the gateway IP at abnormal MAC addresses.

Firewall Logs: Review for unusual ARP traffic patterns or duplicate gateway MAC entries.

NetFlow Analysis: Examine traffic volume shifts; sudden appearance of new MACs forwarding large volumes of traffic.

Softened Interpretation:
The network’s addressing system is its heartbeat. This attack skews it. Triage starts with mapping what should be there – and finding the impostor.

Clarifying Context (Attacker Entry Assumption)
In this case, we observe unsolicited ARP replies originating from a suspicious MAC address, repeatedly claiming ownership of the default gateway IP. Given the prevalence of ARP spoofing attacks initiated from within the network via compromised hosts (e.g., malware infections or insider threats), we must consider the attacker’s initial access method. However, for the purposes of this investigation, evidence strongly suggests that the attacker gained Layer 2 presence through a Wi-Fi breach. This scenario often arises from unsecured or poorly configured wireless networks, where the attacker can join the LAN and inject malicious ARP packets without first compromising a host. Therefore, while the triage focuses on network telemetry (Nmap, NetFlow, firewall), we acknowledge that host-based indicators or physical device checks might be warranted in a complete investigation.



**4. Host Operating System Layer Mapping (If Known)**
ARP spoofing operates purely at Layer 2 of the OSI model, with minimal direct impact on the host system unless the attack escalates (e.g., credential theft via sniffed traffic).

Layer 6 – Network Communication: Host systems may show failed connections, dropped sessions, or traffic routed to an unexpected MAC.

Softened Interpretation:
The host feels the pulse but doesn’t see the hand pulling the strings. It just notices something is off.

**5. Host Operating System Cross-Layer Interaction Pivots**
In this ARP spoofing case, the primary impact occurs at the OSI Layer 2 (Data Link Layer), manipulating network communication. The host operating system itself does not register the ARP spoofing directly. However, if the attack escalates (for example, through credential theft, session hijacking, or malware delivery), potential host system pivots may include:

Layer 6 (Network Communication) → Layer 4 (Credential Management): If credentials are stolen via session hijacking, the attacker may pivot to access privileged host services.

Layer 6 → Layer 2 (Persistence Infrastructure): If malware is delivered through compromised sessions, it could establish persistence mechanisms such as startup entries or scheduled tasks.

Softened Interpretation:
The host’s OS might feel indirect effects—like dropped connections or inconsistent routing—but these are symptoms of a deeper network-layer deception. Without escalation, the OS stays unaware; with it, it may become the next target.


**6. OSI Layer Relevance**
Layer	Relevance
Layer 2 – Data Link	ARP protocol manipulation, MAC spoofing
Layer 3 – Network	Redirection of IP packets
Layer 7 – Application	Interception of application data (e.g., web, email)

Softened Interpretation:
The attack starts with unsolicited ARP messages (Layer 2), disrupts routing decisions (Layer 3), and can ultimately compromise communication sessions and data (Layer 7) if leveraged for escalation.

**7. Attacker Behavior Interpretation (Narrative Format)**
The attacker’s behavior in this ARP spoofing case reveals a mix of opportunism and subtlety. Their first move isn’t to scan for weaknesses directly, but to gain a presence inside the network—for example, by joining an unsecured Wi-Fi network or compromising an internal host. This initial access allows them to capture local traffic at Layer 2.

With their sniffer running (via tools like TCPDump or Wireshark), they observe ARP broadcasts—learning which IP addresses correspond to which MAC addresses. They identify the default gateway (the router handling outbound traffic) and recognize a critical gap: the network lacks ARP monitoring or defenses.


The attacker is probing the network, searching for a weak spot. They notice no ARP monitoring is in place, and the default gateway MAC can be spoofed. By sending repeated ARP replies, they slowly poison the victim’s ARP cache, redirecting traffic through their machine. They’re cautious – watching responses, checking which systems react. Once they confirm control, they capture credentials, sniff traffic, or inject fake responses. This is reconnaissance, foothold, and possibly active compromise – all in one move.

**8. Defender Action Summary (Narrative Format)**
The detection begins with an observation of unsolicited ARP replies in a Wireshark capture—packets originating from a suspicious MAC address and repeatedly asserting ownership of the default gateway IP. This anomalous behavior, coupled with supporting evidence from switch logs showing frequent ARP updates for the same gateway IP, raises immediate red flags. Such unsolicited replies, especially from a device not previously observed as the gateway, suggest active ARP spoofing attempts.

Once this pattern is noticed, defenders pivot to a targeted verification of ARP tables on affected endpoints. Captured ARP caches can be compared against known good MAC-to-IP mappings, focusing particularly on the default gateway. If mismatches are found—where an unexpected MAC claims a critical IP—the conclusion is clear: ARP poisoning is in effect.

At the same time, defenders review switch port logs and configurations, using port security and Dynamic ARP Inspection (DAI) to cross-reference port assignments with legitimate MAC addresses. If port security is misconfigured or missing, this reveals an avenue the attacker exploited to inject rogue ARP replies.

Response actions include isolating the suspicious device (identified by its MAC address) from the network, restoring correct ARP entries via manual static assignments or ARP cache flushing, and configuring switch-level protections like DAI and port security to prevent further unauthorized ARP traffic. If switch logs indicate the MAC originated from an unauthorized Wi-Fi connection, the defender’s focus must shift to Wi-Fi access controls and authentication hardening.

Finally, defenders review monitoring and alerting systems to ensure ARP-related anomalies are promptly flagged in the future. This includes enabling ARP watch utilities, correlating ARP storms or flooding patterns, and aligning SIEM rules to detect high-frequency ARP changes on key network devices.

**9. Attacker Strategy Notes (Narrative Format)**
The attacker’s strategy in this ARP spoofing case is a multi-phase operation, leveraging both physical access and network protocol vulnerabilities to achieve their objectives. Their initial foothold comes not from malware or social engineering, but from gaining unauthorized access to the local network via a compromised Wi-Fi point. This deliberate choice sidesteps traditional perimeter defenses and allows the attacker to operate from within the victim’s broadcast domain, effectively “posing as a neighbor” to the systems they intend to compromise.

With local access established, the attacker pivots to reconnaissance. Rather than conducting loud scans that might trigger alarms, they adopt a quiet approach, passively capturing legitimate ARP traffic using tools like Wireshark or TCPDump. This eavesdropping reveals the network’s MAC-to-IP mappings, notably identifying the default gateway—the device that routes internal traffic to external destinations. Recognizing the absence of ARP monitoring or security measures such as Dynamic ARP Inspection or port security, the attacker sees an opportunity to exploit this gap.

The tactical move is the ARP spoofing attack itself. The attacker sends gratuitous ARP replies—unsolicited messages asserting that their machine’s MAC address owns the IP address of the default gateway. These replies are crafted and spaced carefully, minimizing detection while gradually poisoning the ARP caches of nearby hosts. This misrouting of traffic is subtle, creating a man-in-the-middle position where the attacker can intercept and manipulate data without immediately disrupting services.

This method’s brilliance lies in its low profile and adaptability. The attacker can escalate at their discretion: capturing credentials from misrouted traffic, injecting malicious payloads, or simply observing the victim’s communication patterns for future exploitation. They can also adjust the frequency and nature of their ARP replies to maintain control while avoiding detection by less vigilant monitoring systems.

In essence, the attacker isn’t brute-forcing the network—they’re exploiting its trust and predictability. By masquerading as a legitimate internal node, they leverage the network’s own mechanisms (ARP) against it. Their strategy reflects a nuanced understanding of both the protocol’s inherent weaknesses and the common oversights in network security configurations. It’s not just an attack—it’s a patient manipulation of network dynamics, a silent insertion into the flow of trusted communications.


10. Who’s Who – Object Role Clarification
Object / Term	Role in Attack or Investigation
ARP	Protocol used to resolve IP addresses to MAC addresses
Wireshark	Tool to capture and analyze network traffic
MAC Address	Hardware identifier used at Layer 2
NetFlow	Network flow analysis tool for traffic patterns
Dynamic ARP Inspection	Network defense mechanism against ARP spoofing

11. Conclusion / README

This case study on ARP spoofing via Wi-Fi breach illustrates a classic but frequently underestimated threat vector in network security. The attacker, by first breaching the Wi-Fi perimeter, positions themselves inside the trusted broadcast domain, bypassing traditional edge defenses. From this internal vantage point, they execute a silent yet potent ARP spoofing attack, a technique that poisons the ARP caches of local devices and misroutes their traffic through the attacker’s system.

Our investigation began with the observation of unsolicited ARP replies in packet captures, reinforced by switch logs showing unexpected ARP updates. Recognizing these patterns as indicative of an ARP poisoning attempt, we pivoted to a structured triage approach—first focusing on network telemetry (Wireshark, NetFlow, firewall logs), but soon acknowledging the likely host-based implications and the necessity to consider file system, registry, and volatile memory artifacts if the attack escalates.

Through careful reconstruction, we identified the attacker’s strategic goals: to insert themselves into the network flow undetected, intercept sensitive data, and potentially manipulate traffic in real-time. The gratuitous ARP replies were not random noise but deliberate, spaced attempts to claim control without alerting defenses.

This case reinforces several critical lessons for both CySA+ candidates and practicing defenders:

ARP spoofing is not a relic attack; it remains effective in environments lacking proper defenses like Dynamic ARP Inspection or port security.

Initial access via Wi-Fi breach highlights the importance of strong access controls and network segmentation.

Observability is paramount; even simple tools like packet captures or switch logs can reveal complex attack chains when interpreted correctly.

Cross-layer thinking is essential; what begins as a network-layer event may escalate into host-level compromise, requiring multi-faceted triage and response.

Ultimately, this case is a testament to the attacker’s ability to blend stealth with precision, leveraging protocol vulnerabilities and environmental weaknesses. It reminds defenders that foundational knowledge of network protocols, coupled with systematic triage methodologies, is crucial to early detection and containment.


